In [ ]:
import os
#os.environ['KMP_DUPLICATE_LIB_OK']='True'
import sys
sys.path.append("./../..")

# Load the minimum required library to run the functions
#from Utils_functions import *
from InternalLibrary.StatisticalFunctions import *
from InternalLibrary.SimulatorPackage import Simulator_noGPU

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch

from tqdm import tqdm
import _pickle as pickle
from numba import jit

from scipy.optimize import curve_fit

In [159]:
from pathos.multiprocessing import ProcessingPool as ProcessPool
import pathos

class ParallelSimulator():
    def __init__(self, dt, DeltaT, TotalT, prior_lims, transient_time = 0, batch_size = 50) -> None:
        self.sim_param = {'dt': dt, 'DeltaT': DeltaT, 'TotalT': TotalT, 'transient_time': transient_time}
        self.pool = ProcessPool(nodes=pathos.helpers.cpu_count())
        self.batch_size = batch_size
        self.prior_limits = prior_lims
        _ = self._dummy_run()
    
    def _dummy_run(self):
        Simulator_noGPU(dt = 1, DeltaT=1, TotalT = 2, theta = np.zeros(5).reshape(5,1,1), transient_time = 0)
    
    def _run_simulator(self, theta):
        return Simulator_noGPU(theta = theta, **self.sim_param)
    
    def _get_theta(self):
        return get_theta_from_prior(self.prior_limits, self.batch_size)[0]
    
    def run(self,N):
        thetas = [self._get_theta() for _ in range(N)]
        s = self.pool.map(self._run_simulator, thetas)
        x = s[0][0]
        y = s[0][1]
        f = s[0][2]
        for i in np.arange(1,len(s)):
            x = np.concatenate([x, s[i][0]])
            y = np.concatenate([y, s[i][1]])
            f = np.concatenate([f, s[i][2]])
        t = thetas[0]
        for i in np.arange(1,len(thetas)):
            t = np.concatenate([t, thetas[i]],axis = 1)
        return x, y, f, t
    
@jit(nopython=True)
def ComputeEmpiricalEntropy_J(x_trace, y_trace, f_trace, theta, dt, k_x=6e-3, kbT=4.11):
    '''
    Compute the entropy production for the given traces and parameters
    
    INPUT 
    x_trace: array of shape (n_sim, sampled_point_amount) with the x traces
    y_trace: array of shape (n_sim, sampled_point_amount) with the y traces
    f_trace: array of shape (n_sim, sampled_point_amount) with the f traces
    theta: array of shape (9, n_sim) with the parameters

    OUTPUT
    S_mean: mean entropy production
    Fx: array of shape (n_sim, sampled_point_amount) with the x forces
    Fy: array of shape (n_sim, sampled_point_amount) with the y forces
    S_tot: array of shape (n_sim) with the entropy production for each simulation
    '''
    # Unpack Parameters
    mu_y = theta[0]
    k_y = theta[1]
    k_int = theta[2]
    tau = theta[3]
    eps = theta[4]

    D_y = kbT * mu_y

    # Compute the force
    F_x = - k_x * x_trace + k_int * y_trace
    F_y = - k_y * y_trace + k_int * x_trace + f_trace
    F_xs = (F_x[:,1:] + F_x[:,:-1])/ (2*dt)
    F_ys = (F_y[:,1:] + F_y[:,:-1])/ (2*dt)

    # Compute the entropy production
    S_x = sum((x_trace[:,1:] - x_trace[:,:-1]) * F_xs, axis = 1)
    S_y = sum((y_trace[:,1:] - y_trace[:,:-1]) * F_ys, axis = 1)
    S_tot = S_x + S_y
    
    return S_tot

In [174]:
dt = 1e-6
DeltaT = 1/25_000
TotalT = 20
transient = 0.5
prior_limits = {
    "mu_y": [1e4, 140e4],
    "k_y": [1.5e-2, 30e-2],
    "k_int": [1e-3, 6e-3],
    "tau": [2e-2, 20e-2],
    "eps": [0.5, 6],
}

batch_size = 50
N = pathos.helpers.cpu_count()
TotalSim = batch_size * N

In [180]:
TotalT_range = np.arange(1,20,1)
correlations = np.zeros(len(TotalT_range))
i = 0
for T in TotalT_range:
    Simulator = ParallelSimulator(dt, DeltaT, T, prior_limits, transient_time = 0.5, batch_size = batch_size)
    x, y, f, thetas = Simulator.run(N)
    _, _, _, Emp = ComputeEmpiricalEntropy(x, y, f, thetas, TotalSim, DeltaT)
    Theo,_ = ComputeTheoreticalEntropy(thetas, TotalSim)
    correlations[i] = np.corrcoef(Emp, Theo.reshape(-1))[0,1]
    i += 1

KeyboardInterrupt: 

In [ ]:
fig, ax = plt.subplots(1,1,figsize = (10,10))
ax.plot(TotalT_range, correlations)
ax.set_xlabel("TotalT")
ax.set_ylabel("Correlation")

In [ ]:
# Fit the curve
def func(x, a, b):
    return a + b*x

popt, pcov = curve_fit(func, TotalT_range, correlations)
a,b = popt
print("Fitted Correlation Curve in Time")
print("Function used: a + b*x")
print(f"a: {a}")
print(f"b: {b}")
